# 기획자나 마케터가 되어보자 1탄 
## 이벤트 댓글을 분석해보기

### 댓글 속 제품에 대한 관심을 빈도 수로 추출해야 한다면?
* 쇼핑몰에서 제품 관련 이벤트 진행시 어떤 제품을 고객이 선호하는지?
* 고객 DB와 연계할 수 있다면 이벤트 혹은 마케팅 세그먼트로 활용 가능?

### 그렇게 이어지는 향후 마케팅 전략 수립 

# load library

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import requests #requests는 웹브라우저로 웹사이트 내용을 가져오는 라이브러리
from bs4 import BeautifulSoup as bs #beautifulsoup을 통해 읽어 온 웹페이지 파싱

from tqdm import trange #작업이 얼마나 진행되었는지 확인
%matplotlib inline

In [6]:
base_url="https://www.inflearn.com/pages/newyear-event-20200102"
response=requests.get(base_url) #url내용 받아옴
response #전체 페이지가 내 주피터 노트북에 받아졌다 (reposnce[200]: 정상적으로 받아왔다)

<Response [200]>

In [37]:
#response.text #페이지소스 

In [8]:
#html 형식의 response.text를 파싱해서 내용을 담아줘
soup=bs(response.text,'html.parser')


In [14]:
# main > section > div > div > div.chitchats > div.chitchat-list >
# div:nth-child(33) > div > div.body.edit-chitchat

content=soup.select("#main > section > div > div > div.chitchats > div.chitchat-list > div")
content[-1]

<div class="chitchat-item chitchat-editor-closed comment-editor-closed" data-id="23158">
<figure class="image is-48x48">
<img alt="HONGJOO Ha" class="user_thumb is-rounded" src="https://cdn.inflearn.com/public/main/profile/default_profile.png"/>
</figure>
<div class="content">
<div class="author">
<a class="author_name" href="/users/229263">HONGJOO Ha</a>
<time class="created_at" datetime="Thu Jan 30 2020 20:37:05 GMT+0900 (GMT+09:00)">⋅ 12개월 전</time>
<a class="chitchat__button chitchat__button--update-chitchat by-chitchat-editor-opened" type="button">저장</a>
<a class="chitchat__button chitchat__button--hide-editor by-chitchat-editor-opened" type="button">취소</a>
<a class="chitchat__button chitchat__button--hide-comment-editor by-comment-editor-opened" type="button">작성취소</a>
<a class="chitchat__button chitchat__button_recommend e-like-button" type="button">
<button class="T-like-button" data-reaction="" data-recommended_cnt="0">
<span class="infd-icon fill"><svg viewbox="0 0 14 12" width

In [19]:
chitchat=content[-1].select("div.body.edit-chitchat")[0]
chitchat

<div class="body edit-chitchat by-chitchat-editor-closed">오늘코드 사랑해요 ~ 
데이터 분석 기초를 둔둔하게~ 
#관심강의 : Oracle SQL, 데이터 분석, 파이썬</div>

In [15]:
#원하는 내용만 가져오고 앞뒤 공백은 지워달라
chitchat=content[-1].select("div.body.edit-chitchat")[0].get_text(strip=True) 

In [16]:
chitchat

'오늘코드 사랑해요 ~ \n데이터 분석 기초를 둔둔하게~ \n#관심강의 : Oracle SQL, 데이터 분석, 파이썬'

In [20]:
for i in range(5):
    print("-"*20)
    chitchat=content[i].select("div.body.edit-chitchat")[0].get_text(strip=True) 
    print(chitchat)

--------------------
2020년  목표: 프알못에서 프로젝트 마스터 되기 
관심 있는 강의 : 스프링 , React
--------------------
2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!
관심 있는 강의 : 스프링 웹 MVC
--------------------
2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! 
관심있는 강의: 따배씨 C++
--------------------
2020년 목표 : 리액트 공부하기
--------------------
40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급


In [22]:
content_count=len(content) #게시물 댓글을 content에 담았었는데 개수는?

In [36]:
events=[]
for i in range(content_count):
    #print("-"*20)
    chitchat=content[i].select("div.body.edit-chitchat")[0].get_text(strip=True) 
    #print(chitchat)
    events.append(chitchat)

In [32]:
#events

In [26]:
#trange로 진행률 확인해보기
events=[]
for i in trange(content_count):
    chitchat=content[i].select("div.body.edit-chitchat")[0].get_text(strip=True) 
    events.append(chitchat)

100%|██████████| 100/100 [00:00<00:00, 4248.08it/s]


In [29]:
df=pd.DataFrame({"text":events})
df.shape

(100, 1)

In [30]:
df.to_csv("inflearn-event.csv",index=False)

In [31]:
pd.read_csv("inflearn-event.csv").head()

,text
0,2020년 목표: 프알못에서 프로젝트 마스터 되기 \n관심 있는 강의 : 스프링 ...
1,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
2,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
3,2020년 목표 : 리액트 공부하기
4,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
